# Load Enron emails and store them in a structured dataframe

This notebook load the [Kaggle Enron Email Dataset](https://www.kaggle.com/wcukierski/enron-email-dataset), clean it and export it into a compressed csv file (`../../data/processed/proc_email.csv.zip`) which is going to be used in the other notebooks.

In [1]:
# Import libraries
import os, sys, email
import pandas as pd


In [2]:
# Read the data into a DataFrame
emails_df = pd.read_csv('../../data/local/emails.csv')
print(emails_df.shape)
print(emails_df.head())


(517401, 2)
                       file                                            message
0     allen-p/_sent_mail/1.  Message-ID: <18782981.1075855378110.JavaMail.e...
1    allen-p/_sent_mail/10.  Message-ID: <15464986.1075855378456.JavaMail.e...
2   allen-p/_sent_mail/100.  Message-ID: <24216240.1075855687451.JavaMail.e...
3  allen-p/_sent_mail/1000.  Message-ID: <13505866.1075863688222.JavaMail.e...
4  allen-p/_sent_mail/1001.  Message-ID: <30922949.1075863688243.JavaMail.e...


In [3]:
print(emails_df.shape)
emails_df.head()

(517401, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [4]:
# Display what a single message looks like
print(emails_df['message'][10])

Message-ID: <33076797.1075855687515.JavaMail.evans@thyme>
Date: Mon, 16 Oct 2000 06:42:00 -0700 (PDT)
From: phillip.allen@enron.com
To: buck.buckner@honeywell.com
Subject: Re: FW: fixed forward or other Collar floor gas price terms
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: "Buckner, Buck" <buck.buckner@honeywell.com> @ ENRON
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf

Mr. Buckner,

 For delivered gas behind San Diego, Enron Energy Services is the appropriate 
Enron entity.  I have forwarded your request to Zarin Imam at EES.  Her phone 
number is 713-853-7107.  

Phillip Allen


#### Define reading functions 

In [5]:
## Helper functions
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs


#### Reorganise dataframe into a list 

In [6]:
# Parse the emails into a list email objects
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)
# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages

emails_df.head()

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",(phillip.allen@enron.com),(tim.belden@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",(phillip.allen@enron.com),(leah.arsdall@enron.com),Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",(phillip.allen@enron.com),(randall.gay@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p


In [7]:
emails_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   file                       517401 non-null  object
 1   Message-ID                 517401 non-null  object
 2   Date                       517401 non-null  object
 3   From                       517401 non-null  object
 4   To                         495554 non-null  object
 5   Subject                    517401 non-null  object
 6   Mime-Version               517372 non-null  object
 7   Content-Type               517372 non-null  object
 8   Content-Transfer-Encoding  517372 non-null  object
 9   X-From                     517372 non-null  object
 10  X-To                       517372 non-null  object
 11  X-cc                       517372 non-null  object
 12  X-bcc                      517372 non-null  object
 13  X-Folder                   517372 non-null  

#### Clean the list of emails, keeping only relevant fields

In [8]:
# Set index and drop columns with two few values
emails_df = emails_df.set_index('Message-ID')\
    .drop(['file', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding'], axis=1)
# Parse datetime
emails_df['Date'] = pd.to_datetime(emails_df['Date'], infer_datetime_format=True)
emails_df.dtypes

Date          object
From          object
To            object
Subject       object
X-From        object
X-To          object
X-cc          object
X-bcc         object
X-Folder      object
X-Origin      object
X-FileName    object
content       object
user          object
dtype: object

In [9]:
analysis_df=emails_df[['From', 'To', 'Date','content']].dropna().copy()
analysis_df = analysis_df.loc[analysis_df['To'].map(len) == 1]
analysis_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 354600 entries, <18782981.1075855378110.JavaMail.evans@thyme> to <28618979.1075842030037.JavaMail.evans@thyme>
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   From     354600 non-null  object
 1   To       354600 non-null  object
 2   Date     354600 non-null  object
 3   content  354600 non-null  object
dtypes: object(4)
memory usage: 13.5+ MB


In [10]:
# Save data into a conpressed csv file.
analysis_df.to_csv('../../data/interim/proc_email.csv.zip',compression='zip')


### Now, clean the email content
This stages include:
1. Tokenization
    * Split the text into sentences and the sentences in words
    * transform everything to lowercase
    * remove punctuation
1. Remove all stopwords
1. Lemmatize 
    * change from third person into first person
    * change past and future tense verbs to present tense
    * this makes it possible to combine all words that point to the same thing
1. Stem the words
    * reduce words to their root form
    * e.g. walking and walked to walk


In [11]:
# NLP
import nltk
nltk.download(['stopwords','wordnet'])
from nltk.tokenize.regexp import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/locupe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/locupe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


##### Definition of the cleaning function

In [12]:
def clean(text):
    if(type(text) != str): 
       text = str(text) 
    
    # Define stopwords and punctuation to exclude
    stop = set(stopwords.words('english'))
    stop.update(("to", "cc", "subject", "http", "from", "sent", "ect", "u", "fwd", "www", "com", 'html','pm'))
    exclude = string.punctuation
    
    # Import the lemmatizer from nltk
    lemma = WordNetLemmatizer()
    porter= PorterStemmer()
    
    # Clean the text   
    text = text.rstrip()  # remove whitespace
    text = text.lower() # replace with lowercase
    punc_free = " ".join(word.strip(exclude) for word in text.split())
    stop_free = " ".join([i for i in punc_free.split() if((i not in stop) and (not i.isdigit()))])
    normalized = " ".join(lemma.lemmatize(word) for word in stop_free.split())
    # stem = " ".join(porter.stem(token) for token in normalized.split())
    
    return normalized

In [13]:
# Clean the emails in df and print results
analysis_df['clean_content'] = analysis_df.apply(lambda x: clean(x['content']), axis=1)

In [14]:
analysis_df.info()
# clean_df = analysis_df.drop('content', axis=1)
clean_df = analysis_df

<class 'pandas.core.frame.DataFrame'>
Index: 354600 entries, <18782981.1075855378110.JavaMail.evans@thyme> to <28618979.1075842030037.JavaMail.evans@thyme>
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   From           354600 non-null  object
 1   To             354600 non-null  object
 2   Date           354600 non-null  object
 3   content        354600 non-null  object
 4   clean_content  354600 non-null  object
dtypes: object(5)
memory usage: 16.2+ MB


##### Save the "cleaned" emails in a file

In [15]:
clean_df.to_csv('../../data/processed/clean_email.csv.zip',compression='zip')

In [16]:
clean_df.head()

,From,To,Date,content,clean_content
Message-ID,,,,,
<18782981.1075855378110.JavaMail.evans@thyme>,(phillip.allen@enron.com),(tim.belden@enron.com),2001-05-14 16:39:00-07:00,Here is our forecast\n\n,forecast
<15464986.1075855378456.JavaMail.evans@thyme>,(phillip.allen@enron.com),(john.lavorato@enron.com),2001-05-04 13:51:00-07:00,Traveling to have a business meeting takes the...,traveling business meeting take fun trip espec...
<24216240.1075855687451.JavaMail.evans@thyme>,(phillip.allen@enron.com),(leah.arsdall@enron.com),2000-10-18 03:00:00-07:00,test successful. way to go!!!,test successful way go
<13505866.1075863688222.JavaMail.evans@thyme>,(phillip.allen@enron.com),(randall.gay@enron.com),2000-10-23 06:13:00-07:00,"Randy,\n\n Can you send me a schedule of the s...",randy send schedule salary level everyone sche...
<30922949.1075863688243.JavaMail.evans@thyme>,(phillip.allen@enron.com),(greg.piper@enron.com),2000-08-31 05:07:00-07:00,Let's shoot for Tuesday at 11:45.,let's shoot tuesday 11:45
